숫자 생성 인공지능 개발
GAN은 생성자 신경망과 판별자 신경망 두개를 필요하며, 시퀄셜 형태로 만든다.
LeakyReLU 활성화 함수
keras.optimizers import Adam 학습한 모델의 오차를 줄이기 위해서 경사하강법을 사용
옵티마이저는 Adam을 사용한다.
keras.datasets import mnist 데이터셋중 mnist을 불러온다.
tqdm 모델 학습을 시각적으로 보여주는 라이브러리

In [10]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

mnist 데이터를 불러온다
(x_train, y_train) 훈련 데이터
(x_test, y_test) 검증 데이터
x_test = (x_test.astype(np.float32) - 127.5) / 127.5 10000개로 구성된 x_test를 사용
데이터를 정규화하기 -1 ~ 1 사이의 값으로 나타내고 실수값으로 변경
mnist_data = x_test.reshape(10000,784) 28 X 28 데이터를 1열로 바꾼다.784 X 1 형태로 변환

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = (x_test.astype(np.float32) - 127.5) / 127.5
mnist_data = x_test.reshape(10000,784)
print(mnist_data.shape)
print(len(mnist_data))

생성자 신경망 만들기
생성자 신경망에 노이즈 값을 입력하면 생성자 신경망이 숫자 이미지를 생성
generator.add(Dense(units=256, input_dim=100)) 
    입력하는 값은 100.
    100개 픽셀은 노이즈 값으로 램덤한 값이다.
    이 신경망의 첫번째 층은 256개의 노드로 구성되어 있다.
generator.add(LeakyReLU(0.2)) 첫번째 층의 활성화함수
    함수의 음수값은 특정한 기울기를 0.2로 설정한다.
generator.add(Dense(units=512)) 두번째 층의 512개의 노드를 구성
generator.add(Dense(units=784, activation='tanh')) 활성화 함수는 tanh를 사용한다. 다른 것도 무방
    mnist 데이터가 28 X 28이기 때문에 노드 개수는 784로 한다. 

In [ ]:
def create_generator():
    generator = Sequential()
    generator.add(Dense(units=256, input_dim=100))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(units=784, activation='tanh'))
    return generator

g = create_generator()
g.summary()

판변자 신경망 만들기
생성자 신경망이 만든 이미지가 가짜인지 판별하는 신경망, 훈련되지 않은 신경망으로 다음에 적대적 생성 신경망(GAN)을 통해서 생성자신경망과 판별자 신경망을 학습시킨다.
discriminator.add(Dense(units=512, input_dim=784)) 
    생성자가 만든 데이타가 784개 픽셀로 구성되어 있다.
    이 신경망의 첫번재 층은 512개 노드로 되어있다.
discriminator.add(LeakyReLU(0.2)) 첫번째 층의 활성화함수
    함수의 음수값은 특정한 기울기를 0.2로 설정한다.
discriminator.add(Dense(units=256)) 두번째 층의 256개의 노드를 구성
discriminator.add(Dense(units=1, activation='sigmoid')) 최종 출력값은 1개이다.
    진짜면 1 아니면 0을 출력한다.
    활성화 함수는 sigmoid를 사용한다.
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    판별의 오차값은 이항교차 엔트로피(binary_crossentropy)를 사용한다.
    옵티마이저는 Adam로 학습속도를 0.0002 베타최적화값을 0.5로 설정한다.

In [ ]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units=512, input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dense(units=1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return discriminator

d = create_discriminator()
d.summary()

GAN 생성 함수 만들기
discriminator, generator 판별자, 생성자
discriminator.trainable = False 판별자가 학습하지 못하게 한다.
gan_input = Input(shape=(100,)) 입력한 값을 100개의 값으로 이루어진 데이터
    뒤의 값을 비워두면 실제 데이터의 개수 10000개를 자동으로 넣어준다.
x = generator(gan_input) 생성자 신경망에 작성한 100개의 노이즈 값과 10000개의 데이터를 넣는다.
gan_output = discriminator(x) 생성자가 만든 그림을 보고 판단한 결과를 저장한다.
gan = Model(inputs=gan_input, outputs=gan_output) 적대적 생성 신경망 모델을 만든다.
    inputs 생성자 신경망이 만든 그림, outputs 판별자 신경망이 판단한 결과
gan.compile(loss='binary_crossentropy', optimizer='adam')
    신경망의 오차를 줄이기 위한 방법, 옵티마이저는 Adam

In [ ]:
def create_gan(discriminator, generator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

gan = create_gan(d, g)
gan.summary()

결과 확인 함수 만들기
그림을 화면에 보여주는 코드를 작성한다.
noise = np.random.normal(loc=0,scale=1,size=[100,100])
    생성자에 넣어줄 노이즈값을 만들때, 균일한 값을 생성하도록 정규분포 함수를 사용한다.
    loc=0 평균이 0
    scale=1 평균에서 1만큼 떨어져 있는 값 -1 ~ 1 값을 생성
    size=[100,100] 노이즈를 100개 생성하며, 각각의 노이즈는 숫자 100개 씩 구성되어 있다.
generated_images = generator.predict(noise) predict 함수 사용
    노이즈 값을 신경망에 넣어서 값을 예측
generated_images = generated_images.reshape(100,28,28)
    그림의 형태(28X28)가 아니라 한줄로 만들어진 형태(784)로 그림을 바꾸어 준다.
plt.figure(figsize=(10,10)) 10X10만큼 그림 크기를 정해준다.
for i in range(generated_images.shape[0]): 100개의 그림을 그려주는 반복문
plt.subplot(10,10,i+1) 그림의 위치를 정해준다.
plt.imshow(generated_images[i], interpolation='nearest')
    이미지를 출력하는 함수 각 위치에 어떠한 그림을 넣을지 결정한다.
    interpolation='nearest' 이미지를 출력할 때 각 픽셀을 어떻게 나타낼지 결정한다.
plt.axis('off') 그림 이름을 넣지 않는다.
plt.tight_layout() 지금까지 만들 그림을 보여준다.

In [18]:
def plot_generated_images(generator):
    noise = np.random.normal(loc=0,scale=1,size=[100,100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    
    plt.figure(figsize=(10,10))

    for i in range(generated_images.shape[0]):
        plt.subplot(10,10,i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()

적대적 생성 신경망 훈련시키기
batch_size = 128 한번에 학습시킬 그림의 숫자
epochs = 5000 적대적 생성 신경망을 몇번 반복해서 학습할지 결정한다.
for e in tqdm(range(epochs)): tqdm를 사용해서 반복학습을 진행
noise = np.random.normal(0,1,[batch_size, 100]) 생성자에게 줄 노이즈 값을 만든다.
    loc=0 평균이 0
    scale=1 평균에서 1만큼 떨어져 있는 값 -1 ~ 1 값을 생성
    size=[batch_size,100] 노이즈를 128개 생성하며, 각각의 노이즈는 숫자 100개씩 구성되어 있다.
generated_images = g.predict(noise) 
    생성자 모델에 노이즈를 입력하여 생성자 신경망이 그림을그린 뒤 결과를 저장한다.
image_batch = mnist_data[np.random.randint(low=0,high=mnist_data.shape[0],size=batch_size)]
    인공지능이 학급하도록 데이터셋 10000개에서 128개만 램덤으로 뽑는다.
X = np.concatenate([image_batch,generated_images])
    진짜 그림과 생성한 그림을 합친다. 진짜 그림 128개 뒤로 생성그림 128개를 한줄로 만든다.
y_dis = np.zeros(2*batch_size)
    판별자에게 전달한 결과값을 만든다. 
y_dis[:batch_size] = 1 앞의 128번째 이미지까지는 진짜니까 1로 넣는다.
    뒤에 나오는 128개는 생성이미지이다.
d.trainable = True 훈련하도록 한다.
d.train_on_batch(X, y_dis) 훈련된 값을 체크하여 저장한다.
y_gen = np.ones(batch_size) gan에 넣어줄 값을 1로 만든다.
d.trainable = False 더 학습하지 못하게 한다.
gan.train_on_batch(noise,y_gen) gan에 노이즈값을 넣고, 
plot_generated_images(g) 생성자가 만든 그림을 보여준다.

In [ ]:
batch_size = 128
epochs = 5000

for e in tqdm(range(epochs)):
    noise = np.random.normal(0,1,[batch_size, 100])
    generated_images = g.predict(noise)
    image_batch = mnist_data[np.random.randint(low=0,high=mnist_data.shape[0],size=batch_size)]
    X = np.concatenate([image_batch,generated_images])
    y_dis = np.zeros(2*batch_size)
    y_dis[:batch_size] = 1
    d.trainable = True
    d.train_on_batch(X, y_dis)
    noise = np.random.normal(0,1,[batch_size,100])
    y_gen = np.ones(batch_size)
    d.trainable = False
    gan.train_on_batch(noise,y_gen)

    if e == 0 or e % 1000 == 0:
        plot_generated_images(g)